# Le Debruiteur
* Jonas Freiburghaus
* Romain Capocasale
* He-Arc, INF3dlm-a
* Image Processing course
* 2019-2020

# Import

In [4]:
%matplotlib inline

from debruiteur.generator.datagenerator import DataGenerator
from debruiteur.models.autoencoders import build_conv_autoencoder, build_dense_autoencoder
from debruiteur.models.gan import GAN
from debruiteur.noise.noise import *
from debruiteur.plots.plots import plot_model_loss
from debruiteur.preprocessing.preprocessor import *
from debruiteur.utils.utils import save_model, split_train_val_df
from debruiteur.noise.noise_reduction import * 
from debruiteur.statistics.statistics import * 

import os

# Configurations

In [5]:
noise_class_list = [
    GaussianNoise(mean=0, std=10),
    UniformNoise(amplitude=100),
]

# Create Dataframe

In [6]:
working_dir = os.path.abspath(os.getcwd())

df_original = make_original_dataframe(os.path.join(working_dir, "images"))
df_resized = make_resized_dataframe(df_original, img_shape=(64, 64, 1), resized_path=os.path.join(working_dir, "resized_images"))
df_noised = make_noised_dataframe(df_resized, noise_class_list, os.path.join(working_dir, "noised_images"))
df_train, df_val = split_train_val_df(df_noised)

# Models training

## Data generator

In [ ]:
train_gen = DataGenerator(df_train)
val_gen = DataGenerator(df_val)

## Convolutional autoencoder

In [ ]:
ae_model = build_conv_autoencoder()
ae_model.summary()

In [ ]:
history = ae_model.fit_generator(train_gen, validation_data=val_gen, epochs=10)
plot_model_loss(history)

In [ ]:
save_model(ae_model, os.path.join(working_dir, "saved_models"), "conv_autoencoder.h5")

del ae_model
del train_gen
del val_gen

## Dense autoencoder

In [ ]:
train_gen = DataGenerator(df_train, img_shape=(10000, ))
val_gen = DataGenerator(df_val, img_shape=(10000, ))

In [ ]:
ae_model = build_dense_autoencoder(loss="mse")
ae_model.summary()

In [ ]:
history = ae_model.fit_generator(train_gen, validation_data=val_gen, epochs=10)
plot_model_loss(history)

In [ ]:
save_model(ae_model, os.path.join(working_dir, "saved_models"), "dense_autoencoder.h5")

del ae_model
del train_gen
del val_gen

## GAN autoencoder

In [ ]:
train_gen = DataGenerator(df_train, img_shape=(64, 64, 1), batch_size=8)
val_gen = DataGenerator(df_val, img_shape=(64, 64, 1), batch_size=8)

In [ ]:
gan = GAN(img_shape=(64, 64, 1))

In [ ]:
gan.generator.summary()

In [ ]:
gan.discriminator.summary()

In [ ]:
train_hist, val_hist = gan.train(train_gen, val_gen, batch_size=8)

# Statistics

In [7]:
noise_reduction_methods = [('Wiener Filter', wiener_filter),
                   ('Laplacian Filter', laplacian_filter),
                   ('Gaussian Weighted Filter', gaussian_weighted_substract_filter),
                   ('Mean Filter', mean_filter),
                   ('Median Filter', median_filter),
                   ('Conservative Filter', conservative_filter),
                   ('FFT FIlter', fft_filter)]

In [8]:
compute_noise_reduction_method_statistics(df_val[0:10],noise_reduction_methods)

C:\Users\Romain\Desktop\ProjetHeArc\ledebruiteur\debruiteur\metrics\metrics.py:16: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im_true.
  psnr = metrics.peak_signal_noise_ratio(orignal_img, transformed_img)
C:\ProgramData\Anaconda3\lib\site-packages\skimage\metrics\_structural_similarity.py:108: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  im2[..., ch], **args)



Compute finish for Wiener Filter



Compute finish for Laplacian Filter



Compute finish for Gaussian Weighted Filter



Compute finish for Mean Filter



Compute finish for Median Filter



Compute finish for Conservative Filter



Compute finish for FFT FIlter


,MSE,NRMSE,PSNR,SSIM
Wiener Filter,2.419304e+04,0.998265,4.913054,0.000328
Laplacian Filter,1.313799e+04,0.717286,7.918620,-0.019731
Gaussian Weighted Filter,9.458806e+03,0.585890,10.077640,0.242368
Mean Filter,9.502000e+03,0.594020,9.881130,0.131138
Median Filter,9.576905e+03,0.596763,9.835331,0.145576
Conservative Filter,9.247270e+03,0.578047,10.243448,0.238477
FFT FIlter,3.435418e+12,10963.188712,-74.686032,0.000003


In [10]:
compute_noise_type_statistics(df_resized[0:10], noise_reduction_methods, noise_class_list)


Compute finish for Wiener Filter



Compute finish for Laplacian Filter



Compute finish for Gaussian Weighted Filter



Compute finish for Mean Filter



Compute finish for Median Filter



Compute finish for Conservative Filter



Compute finish for FFT FIlter


,GaussianNoise,UniformNoise
Wiener Filter,1.579149e+04,1.579598e+04
Laplacian Filter,9.193164e+04,1.929703e+04
Gaussian Weighted Filter,1.295343e+04,3.474751e+03
Mean Filter,7.322911e+03,3.167893e+03
Median Filter,1.190029e+04,3.173680e+03
Conservative Filter,1.237780e+04,2.873524e+03
FFT FIlter,1.644942e+13,1.148841e+13
